In [3]:
import os, json, math, asyncio
import shutil
import zarr
import numpy as np
import tensorstore as ts
from tqdm.notebook import tqdm
import time
from rich.pretty import pprint

import pff  # same helper module used in the notebook

def _infer_dp_from_name(pff_path: str):
    parts = pff.parse_name(os.path.basename(pff_path))
    if not parts or 'dp' not in parts:
        raise ValueError("Could not parse data product from filename.")
    dp = parts['dp']
    bpp = int(parts.get('bpp', 2))
    if dp == 'img16':
        img_shape = (32, 32); dtype = np.uint16; bytes_per_image = 32*32*2
        header_kind = 'img_two_level'
    elif dp == 'img8':
        img_shape = (32, 32); dtype = np.uint8; bytes_per_image = 32*32*1
        header_kind = 'img_two_level'
    elif dp == 'ph256':
        img_shape = (16, 16); dtype = np.int16; bytes_per_image = 16*16*2
        header_kind = 'ph256_one_level'
    elif dp == 'ph1024':
        img_shape = (32, 32); dtype = np.int16; bytes_per_image = 32*32*2
        header_kind = 'img_two_level'
    else:
        raise ValueError(f"Unsupported data product type: {dp}")
    return parts, dp, bpp, img_shape, dtype, bytes_per_image, header_kind

def _zarr3_codec_chain(codec: str, level: int):
    allowed_codecs = ["zstd", "gzip", "blosc-lz4"]
    codecs = [{"name": "bytes", "configuration": {"endian": "little"}}]
    if codec == "zstd":
        codecs.append({"name": "zstd", "configuration": {"level": int(level)}})
                
    if codec == "gzip":
        codecs.append({"name": "gzip", "configuration": {"level": int(level)}})
                
    if codec == "blosc-lz4":
        codecs.append({"name": "blosc", "configuration": {"cname": "lz4", "clevel": int(level), "shuffle": "shuffle"}})

    if codec not in allowed_codecs:
        raise ValueError("Unknown codec; use 'zstd', 'gzip', or 'blosc-lz4'.")
    else:
        return codecs

async def _open_ts_array(root_path, name, shape, chunks, np_dtype, codec_chain, attributes=None, create=True, delete_existing=True):
    
    spec = {
        "driver": "zarr3",
        "kvstore": {"driver": "file", "path": str(root_path)},
        "metadata": {
            "shape": list(shape),
            "chunk_grid": {"name": "regular", "configuration": {"chunk_shape": list(chunks)}},
            "data_type": np.dtype(np_dtype).name,
            "codecs": codec_chain,
            "attributes": attributes or {},
            "dimension_names": attributes['_ARRAY_DIMENSIONS']
        },
        "path": name,
        "create": create,
        "open": True,
    }
    # print(spec)
    return await ts.open(spec)

async def convert_pff_to_tensorstore(
    pff_path: str,
    zarr_root: str,
    codec: str = "zstd",
    level: int = 3,
    time_chunk: int = 8192,
):
    if not os.path.exists(pff_path):
        raise FileNotFoundError(pff_path)

    # ensure  target directory is clean
    if os.path.exists(zarr_root):
        shutil.rmtree(zarr_root)

    # create the zarr group using the zarr library.
    #automatically writes the root zarr.json file.
    root_group = zarr.open_group(zarr_root, mode='w')
    print(f"Created Zarr group at: {zarr_root}")

    name_parts, dp, bpp, img_hw, np_img_dtype, bytes_per_image, header_kind = _infer_dp_from_name(pff_path)

    with open(pff_path, "rb") as f:
        i0, nframes, t0, t1 = pff.img_info(f, bytes_per_image)

    codecs = _zarr3_codec_chain(codec, level)

    T = nframes
    H, W = img_hw
    img_shape = (T, H, W)
    img_chunks = (time_chunk, H, W)
    ts_shape_only = (T,)
    ts_chunks_scalar = (max(1024, time_chunk*2),)

    # make arrays within the group using tensorstore.
    images = await _open_ts_array(
        zarr_root, "images",
        img_shape, img_chunks, np_img_dtype, codecs,
        attributes={
            "_ARRAY_DIMENSIONS": ["time", "y", "x"], # for xarray to understand the zarr format
            "source_pff_file": os.path.basename(pff_path),
            "pff_metadata": name_parts,
        }, create=True)

    timestamps = await _open_ts_array(
        zarr_root, "timestamps",
        ts_shape_only, (max(1024, time_chunk*2),), np.float64, codecs,
        attributes={"_ARRAY_DIMENSIONS": ["time"]}, create=True)

    # header schema arrays
    header_arrays = {}
    root_group.create_group('headers')
    if header_kind == "ph256_one_level":
        
        # one-level fields
        header_fields = {
            "quabo_num": np.uint8,
            "pkt_num": np.int64,
            "pkt_tai": np.int64,
            "pkt_nsec": np.int64,
            "tv_sec": np.int64,
            "tv_usec": np.int64,
        }
        for k, dt in header_fields.items():
            header_arrays[k] = await _open_ts_array(
                zarr_root, f"headers/{k}", ts_shape_only, ts_chunks_scalar, dt, codecs,
                attributes={"_ARRAY_DIMENSIONS": ["time"]}, create=True)
    else:
        per_quabo_fields = {
            "pkt_num": np.int64, "pkt_tai": np.int64, "pkt_nsec": np.int64,
            "tv_sec": np.int64, "tv_usec": np.int64,
        }
        for qi in range(4):
            root_group.create_group(f'headers/quabo_{qi}')
            for k, dt in per_quabo_fields.items():
                name = f"headers/quabo_{qi}/{k}"
                header_arrays[name] = await _open_ts_array(
                    zarr_root, name, ts_shape_only, ts_chunks_scalar, dt, codecs,
                    attributes={"_ARRAY_DIMENSIONS": ["time"]}, create=True)

    # batched write buffers
    batch_T = time_chunk
    img_batch = np.empty((batch_T, H, W), dtype=np_img_dtype)
    ts_batch = np.empty((batch_T,), dtype=np.float64)
    header_batches = {k: np.empty((batch_T,), dtype=arr.dtype.numpy_dtype) for k, arr in header_arrays.items()}

    pff_file_size = os.path.getsize(pff_path)  # bytes

    start = time.monotonic()

    # progress bar setup
    pbar = tqdm(total=T, desc="Converting PFF -> Zarr (TensorStore)", unit="frames")
    
    # streaming parse/write
    written = 0
    futures = []

    def flush_batch(n):
        nonlocal written, futures
        if n == 0:
            return
        s = slice(written, written + n)
        f = images[s, :, :].write(img_batch[:n])
        if f is not None:
            futures.append(f)
        f = timestamps[s].write(ts_batch[:n])
        if f is not None:
            futures.append(f)
        for k, arr in header_arrays.items():
            f = arr[s].write(header_batches[k][:n])
            if f is not None:
                futures.append(f)
        written += n
        pbar.update(n)

    with open(pff_path, "rb") as f:
        bi = 0
        for i in range(T):
            hdr_str = pff.read_json(f)
            if hdr_str is None:
                break
            header = json.loads(hdr_str)

            # read image
            flat = pff.read_image(f, W, bpp)
            if flat is None:
                continue
            img_batch[bi, :, :] = np.asarray(flat).reshape(H, W).astype(np_img_dtype)

            # timestamp
            #(TODO: this is not right for now, update with proper time code later)
            if header_kind == "ph256_one_level":
                if "pkt_tai" in header and "pkt_nsec" in header:
                    ts_batch[bi] = float(header["pkt_tai"]) + float(header["pkt_nsec"])*1e-9
                elif "tv_sec" in header and "tv_usec" in header:
                    ts_batch[bi] = float(header["tv_sec"]) + float(header["tv_usec"])*1e-6
                else:
                    ts_batch[bi] = np.nan
                for fld in ("pkt_num", "pkt_tai", "pkt_nsec", "tv_sec", "tv_usec", "quabo_num"):
                    if fld in header_arrays:
                        v = header.get(fld, 0)
                        header_batches[fld][bi] = header_fields[fld](v)
                        # print(header_fields[fld](v))
            else: # two-level (32x32)
                if "tv_sec" in header and "tv_usec" in header:
                    ts_batch[bi] = float(header["tv_sec"]) + float(header["tv_usec"])*1e-6
                else:
                    q0 = header.get("quabo_0", {})
                    if "pkt_tai" in q0 and "pkt_nsec" in q0:
                        ts_batch[bi] = float(q0["pkt_tai"]) + float(q0["pkt_nsec"])*1e-9
                    else:
                        ts_batch[bi] = np.nan

                for qi in range(4):
                    q = header.get(f"quabo_{qi}", {})
                    for fld in ("pkt_num", "pkt_tai", "pkt_nsec", "tv_sec", "tv_usec", "quabo_num"):
                        name = f"headers/quabo_{qi}/{fld}"
                        if name in header_arrays:
                            header_batches[name][bi] = np.int64(q.get(fld, header.get(fld, 0))) # Fallback to top-level

            bi += 1
            if bi == batch_T:
                flush_batch(bi)
                bi = 0

        if bi > 0:
            flush_batch(bi)

    # flush all writes
    for fut in futures:
        await fut

    pbar.close()
    
    end = time.monotonic()
    elapsed_s = end - start

    # attach some root-level attributes using a tiny sidecar array’s attributes or by creating a 0-d array
    # in Zarr v3, array attributes live per node. for a root-level note, put them on images
    spec = images.spec().to_json()

    # compute Zarr store size (sum of files under root)
    def _dir_size(path):
        total = 0
        for dirpath, _, filenames in os.walk(path):
            for fn in filenames:
                fp = os.path.join(dirpath, fn)
                if os.path.exists(fp):
                    total += os.path.getsize(fp)
        return total

    zarr_size = _dir_size(zarr_root)
    compression_ratio = (pff_file_size / zarr_size) if zarr_size > 0 else float("inf")

    # print compression report etc.
    report = {
        "source_pff": os.path.basename(pff_path),
        "frames": T,
        "elapsed_seconds": elapsed_s,
        "pff_size_bytes": pff_file_size,
        "zarr_size_bytes": zarr_size,
        "compression_ratio_pff_over_zarr": compression_ratio,
        "images": {
            "shape": (T, H, W),
            "dtype": str(np.dtype(np_img_dtype)),
            "chunks": (time_chunk, H, W),
        },
        "timestamps": {
            "shape": (T,),
            "dtype": "float64",
            "chunks": (max(1024, time_chunk*2),),
        },
        "header_arrays": sorted(list(header_arrays.keys())),
        "codec": codec,
        "level": level,
    }

    # pretty print report
    pprint(report)
    
    return report

In [4]:
%autoawait asyncio

from pathlib import Path

l0_dir = Path('/mnt/beegfs/data/L0')
l1_dir = Path('/mnt/beegfs/data/L1')

run_dir = 'obs_Lick.start_2024-07-25T04:34:06Z.runtype_sci-data.pffd'
pff_base = "start_2024-07-25T05:27:22Z.dp_img16.bpp_2.module_1.seqno_2"

pff_fname = f'{pff_base}.pff'
pff_path = l0_dir / run_dir / pff_fname
# "start_2024-07-25T04_34_46Z.dp_img16.bpp_2.module_1.seqno_0.debug_TRUNCATED.pff",
# "start_2024-07-25T04_34_46Z.dp_ph256.bpp_2.module_1.seqno_0.debug_TRUNCATED.pff",
# "start_2025-08-04T06:16:24Z.dp_ph256.bpp_2.module_254.seqno_0.pff",
# "start_2025-08-23T06_11_24Z.dp_ph1024.bpp_2.module_254.seqno_0.pff",

zarr_out_path = l1_dir / run_dir / f"{pff_base}.zarr"
assert os.path.exists(pff_path)
os.makedirs(zarr_out_path, exist_ok=True)

original_mask = os.umask(0o000)   # propagate ACL settings from the parent directory to the reduced data (poor man's beegfs acl)???
try:
    ret = await convert_pff_to_tensorstore(
        pff_path,
        zarr_out_path,
        time_chunk=8192,
        codec="zstd",
        # codec="blosc-lz4",
        level=10
    )
finally:
    os.umask(original_mask)

Created Zarr group at: /mnt/beegfs/data/L1/obs_Lick.start_2024-07-25T04:34:06Z.runtype_sci-data.pffd/start_2024-07-25T05:27:22Z.dp_img16.bpp_2.module_1.seqno_2.zarr


Converting PFF -> Zarr (TensorStore):   0%|          | 0/1574804 [00:00<?, ?frames/s]

{
│   'source_pff': 'start_2024-07-25T05:27:22Z.dp_img16.bpp_2.module_1.seqno_2.pff',
│   'frames': 1574804,
│   'elapsed_seconds': 298.1713873851113,
│   'pff_size_bytes': 4000002160,
│   'zarr_size_bytes': 2003629472,
│   'compression_ratio_pff_over_zarr': 1.9963781806459673,
│   'images': {'shape': (1574804, 32, 32), 'dtype': 'uint16', 'chunks': (8192, 32, 32)},
│   'timestamps': {'shape': (1574804,), 'dtype': 'float64', 'chunks': (16384,)},
│   'header_arrays': [
│   │   'headers/quabo_0/pkt_nsec',
│   │   'headers/quabo_0/pkt_num',
│   │   'headers/quabo_0/pkt_tai',
│   │   'headers/quabo_0/tv_sec',
│   │   'headers/quabo_0/tv_usec',
│   │   'headers/quabo_1/pkt_nsec',
│   │   'headers/quabo_1/pkt_num',
│   │   'headers/quabo_1/pkt_tai',
│   │   'headers/quabo_1/tv_sec',
│   │   'headers/quabo_1/tv_usec',
│   │   'headers/quabo_2/pkt_nsec',
│   │   'headers/quabo_2/pkt_num',
│   │   'headers/quabo_2/pkt_tai',
│   │   'headers/quabo_2/tv_sec',
│   │   'headers/quabo_2/tv_usec',
│   │   'headers/quabo_3/pkt_nsec',
│   │   'headers/quabo_3/pkt_num',
│   │   'headers/quabo_3/pkt_tai',
│   │   'headers/quabo_3/tv_sec',
│   │   'headers/quabo_3/tv_usec'
│   ],
│   'codec': 'zstd',
│   'level': 10
}

In [5]:
ds_clean = xr.open_datatree(
    'ph_5s.zarr',
    consolidated=False,
    engine='zarr',
    chunks={},
    # cache=True
)
# ds_rechunked = ds_clean.chunk({'time': 10000,})
ds_clean.images

NameError: name 'xr' is not defined

In [7]:
p5 = ds_clean.images.min(dim='time').mean().compute()
p95 = ds_clean.images.max(dim='time').mean().compute()
print(f"{p5=}, {p95=}")

p5=<xarray.DataArray 'images' ()> Size: 8B
array(280.87695312), p95=<xarray.DataArray 'images' ()> Size: 8B
array(740.83007812)


In [8]:
ds_clean.images[0]

<xarray.DataArray 'images' (y: 16, x: 16)> Size: 2kB
dask.array<getitem, shape=(16, 16), dtype=float64, chunksize=(16, 16), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x

In [9]:
import matplotlib.pyplot as plt

plt.imshow(ds_clean.images[max_trigger_idx+1])

NameError: name 'max_trigger_idx' is not defined